In [ ]:
import matplotlib as pyplot
import pandas as pd
import numpy as np
from collections import defaultdict
import networkx as nx
%matplotlib inline
adj=defaultdict(list)
bet=defaultdict(list)
print("Enter filename")
filename=input()
data=np.loadtxt(filename,delimiter=',')
print(data)
print("no. of vertex: ",np.amax(data))
X=data[:,0]
y=data[:,1]
#print(X)
#print(y)
print("no.of edges: ",len(X))
extra_label=0
be_tree=0
be_family=0
edge_swap=0
vertex_split=0
label_swap=0
visited_edge_list=defaultdict(int)
family=defaultdict(int)
noe=0
nov=int(np.amax(data))
noe=int(len(X))
#print(noe)
G1=nx.Graph()
for i in range(0,noe):
    #a1,b1=map(int,input().strip().split())
    a1=int(data[i,0])
    b1=int(data[i,1])
    G1.add_edge(a1,b1)
#    print("vertex1: ",a1," vertex2: ",b1)
    adj[a1].append(b1)
    adj[b1].append(a1)
#for i in range(1,nov+1):
#    print("adjacency list of: ",i)
#    print(adj[i])
nx.draw(G1,pos=nx.spring_layout(G1,scale=4),edge_color='red',with_labels=True)

visited=[]
visited_edge_count=0

Enter filename


In [ ]:
adj_int=defaultdict(list)
res_list=[]
vis_int=defaultdict(int)
def dfs(origin):
    global visited
    global adj
    global visited_edge_count
    global bet
    global visited_edge_list
    visited[origin]=0
    for i in range(0,len(adj[origin])):
        if(visited[adj[origin][i]]==-1):
            visited_edge_count+=1
            bet[0].append((origin,adj[origin][i]))
            visited_edge_list[(origin,adj[origin][i])]=5
            visited_edge_list[(adj[origin][i],origin)]=5
            dfs(adj[origin][i])
def dfspath(node,dest_node):
    global adj_int
    global res_list
    global vis_int
    if(node==dest_node):
        ret=1
        return ret 
    vis_int[node]=1
    #print("current node: ", node)
    for i in range(0,len(adj_int[node])):
        if(vis_int[adj_int[node][i]]!=1):
            res_list.append(adj_int[node][i])
            y=dfspath(adj_int[node][i],dest_node)
            if(y==0):
                res_list.pop()
            else:
                return y
    return 0
    
    
def find_path(a,b,i):
    global adj_int
    global res_list
    global vis_int
    global bet
    global visited_edge_list
    adj_int.clear()
    res_list.clear()
    vis_int.clear()
    #print("inside find_path: be tree: ",i)
    for index in range(0,len(bet[i])):
        ai=bet[i][index][0]
        bi=bet[i][index][1]
        if(visited_edge_list[(ai,bi)]==5+i):
            adj_int[ai].append(bi)
            adj_int[bi].append(ai)
    #print(adj_int)
    res_list.append(a)
    x=dfspath(a,b)
    return res_list
            
    

In [ ]:
for i in range(0,100000):
    visited.append(-1)
for i in range(1,nov+1):
    if(visited[i]==-1):
#        print(i)
        dfs(i)
#print(visited_edge_list)
print("After DFS the dfs contain edges:")
print(bet[0])
curr_betree=0
while(visited_edge_count<noe):
    print("no of visited edge count till now: ", visited_edge_count)
    if(visited_edge_count==noe):
        break
    curr_betree+=1
    for i in range(1,nov+1):
        for j in range(0,len(adj[i])):
            if visited_edge_list[(i,adj[i][j])]==0:
                #print("edge: (",i,adj[i][j],") not found in visited")
                #print("curr_betree: ", curr_betree)
                #print("visited value of i: ",visited[i], "visited value of adj[i][j]: ",visited[adj[i][j]])
                if(visited[adj[i][j]]<curr_betree or visited[i]<curr_betree):
                    visited_edge_count+=1
                    visited_edge_list[(i,adj[i][j])]=5+curr_betree
                 #   print("edge (",i,",",adj[i][j],") included as well as its vice versa")
                    visited_edge_list[(adj[i][j],i)]=5+curr_betree
                    included=0
                
                    if(curr_betree>=2):
                        a=i
                  #      print("Since current bettree is greater than 2")
                        b=adj[i][j]
                        path_list=[]
                        path_list=find_path(a,b,curr_betree-2)
                   #     print("path from above index: ",path_list)
                        
                        for k in range(0,len(path_list)-1):
                            prev=path_list[k]
                            next1=path_list[k+1]
                            if(visited[prev]<curr_betree-1 or visited[next1]<curr_betree-1):
                               #if(any of the vertex is betree-1 or betree it should not be changed to betree-1 and hence first and condition)
                                if(visited[prev]<curr_betree-1 and visited[next1]<curr_betree-1):
                                    visited[prev]=curr_betree-1
                                    visited[next1]=curr_betree-1
                                elif(visited[prev]<curr_betree-1):
                                    visited[prev]=curr_betree-1
                                elif(visited[next1]<curr_betree-1):
                                    visited[next1]=curr_betree-1
                    #            print(" edge ready for swap : (",prev,",",next1,")")
                                visited_edge_list[(i,adj[i][j])]=5+curr_betree-2
                                visited_edge_list[(adj[i][j],i)]=5+curr_betree-2
                                visited_edge_list[(prev,next1)]=5+curr_betree-1
                                visited_edge_list[(next1,prev)]=5+curr_betree-1
                                bet[curr_betree-2].append((i,adj[i][j]))
                                bet[curr_betree-1].append((next1,prev))
                                included=1
                                edge_swap+=1
                                break
                    
                    if(included==0):
                        bet[curr_betree].append((i,adj[i][j]))
                        if(visited[adj[i][j]]<curr_betree):
                            visited[adj[i][j]]+=1
                        if(visited[i]<curr_betree):
                            visited[i]+=1
print("no of edge_swap: ", edge_swap)
degree=[[0 for i in range(0,nov+1)]for j in range(0,curr_betree+1)]
#print(degree)
G=nx.Graph()
color_ind=-1
for ind in range(0,curr_betree+1):
    if(len(bet[ind])!=0):
        be_tree+=1
        color_ind+=1
        print("Edgelist in betree index: ",ind)
    for col in range(0,len(bet[ind])):
        if(visited_edge_list[bet[ind][col]]==5+ind):
            print("Edge : ", bet[ind][col])
            if(color_ind==0):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='r')
            elif(color_ind==1):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='b')
            elif(color_ind==2):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='g')
            elif(color_ind==3):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='k')
            elif(color_ind==4):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='w')
            elif(color_ind==5):
                G.add_edge(bet[ind][col][0],bet[ind][col][1],color='m')
            degree[ind][bet[ind][col][0]]+=1
            degree[ind][bet[ind][col][1]]+=1
pos=nx.spring_layout(G,k=0.15,iterations=20)
edges=G.edges()
colors=[G[u][v]['color'] for u,v in edges]
nx.draw(G,pos,edges=edges,edge_color=colors,with_labels=True)
#Steps to find family of each vertex



In [ ]:
#print(degree)
curr_maxfamily=[]
for i in range(0,be_tree):
    curr_maxfamily.append(2)
for i in range(0,be_tree):
    for j in range(len(bet[i])):
        if(visited_edge_list[bet[i][j]]==5+i):
            #print("inside family loop")
            li=bet[i][j][0]
            lj=bet[i][j][1]
            if(family[(li,i)]==0 and family[(lj,i)]==0):
                family[(li,i)]=curr_maxfamily[i]
                family[(lj,i)]=curr_maxfamily[i]
                curr_maxfamily[i]+=1
            elif(family[(li,i)]==0):
                family[(li,i)]=family[(lj,i)]
            elif(family[(lj,i)]==0):
                family[(lj,i)]=family[(li,i)]
            else:
                x11=min(family[(li,i)],family[(lj,i)])
                family[(li,i)]=x11
                family[(lj,i)]=x11
    for j in range(len(bet[i])):
        if(visited_edge_list[bet[i][j]]==5+i):
           # print("inside family loop")
            li=bet[i][j][0]
            lj=bet[i][j][1]
            if(family[(li,i)]==0 and family[(lj,i)]==0):
                family[(li,i)]=curr_maxfamily[i]
                family[(lj,i)]=curr_maxfamily[i]
                curr_maxfamily[i]+=1
            elif(family[(li,i)]==0):
                family[(li,i)]=family[(lj,i)]
            elif(family[(lj,i)]==0):
                family[(lj,i)]=family[(li,i)]
            else:
                x11=min(family[(li,i)],family[(lj,i)])
                family[(li,i)]=x11
                family[(lj,i)]=x11

#print(family)                
family_pendant=defaultdict(int)
#print(be_tree)
pendant=[set() for x in range(0,be_tree-1)]
#for i in range(0,be_tree-1):
#    print("pendant: ",i," ", pendant[i])
for i in range(be_tree-1,0,-1):
#    print("in betree: ",i)
    for j in range(1,nov+1):
        if(degree[i][j]==1 and degree[i-1][j]==1):
            if(family_pendant[(i,family[(j,i)])]==0):
                family_pendant[(i,family[(j,i)])]=1
                pendant[i-1].add(j)
#                print("family", family[(j,i)]," ")
#                print("pendant: ",i-1," ", pendant[i-1])
#print(family_pendant)
#print(" now in any node as pendant")
for i in range(be_tree-1,0,-1):
#    print("in betree: ",i)
    for j in range(1,nov+1):
        if(family_pendant[(i,family[(j,i)])]==0):
            if(degree[i][j]!=0 and degree[i-1][j]!=0):
#                print("family", family[(j,i)]," ")
                pendant[i-1].add(j)
#                print("pendant: ",i-1," ", pendant[i-1])
                family_pendant[(i,family[(j,i)])]=1
#                print(family_pendant[(i,family[(j,i)])])
  
                                            

for i in range(0,be_tree-1):
    print("pendant vertices: for be-tree-",i," ", pendant[i])
vertex_rename=[[0 for i in range(0,nov+1)]for j in range(0,curr_betree+1)]
#print(degree)
curr_num=nov+1
#kisi vertex ka agar degree3 hai yaani uss vertex pe degree 2 edge aur degree1 edge bhi hsi kyunki hmlog back edge ki baat krte hai
for i in range(1,nov+1):
    for j in range(0,be_tree-1):
        if(i in pendant[j]):
#            print(" vertex: ",i," is in pendant: ",j)
            vertex_rename[j+1][i]=-2
            continue
        if(degree[j][i]!=0 and degree[j+1][i]!=0):
            vertex_rename[j+1][i]=curr_num
            curr_num+=1
            vertex_split+=1
    for j in range(1,be_tree+1):
        if(degree[0][i]!=1):
            if(vertex_rename[be_tree-j][i]!=0 and degree[be_tree-j][i]==1 and vertex_rename[be_tree-j][i]!=-2):
                vertex_rename[0][i]=vertex_rename[be_tree-j][i]
                vertex_rename[be_tree-j][i]=-1
                label_swap+=1
                break
extra_labels=0
for j in range(1,nov+1):
    for i in range(0,be_tree):
        if(vertex_rename[i][j]==-2):
            vertex_rename[i][j]=vertex_rename[0][j]
print("Total number of vertex finally after all partition : ",curr_num-1)
"""
for i in range(1,nov+1):
    for j in range(1,be_tree-1):
        if(degree[j][i]==1 and i not in pendant[j-1] and i not in pendant[j]):
            extra_labels+=1
for i in range(1,nov+1):
    if(degree[0][i]==1 and i not in pendant[0]):
        extra_labels+=1
    elif(degree[be_tree-1][i]==1 and i not in pendant[be_tree-2]):
        extra_labels+=1
"""
final_degree=[]
final_parent=[]
final_visited=[]
for i in range(0,curr_num):
    final_degree.append(0)
    final_parent.append(-1)
    final_visited.append(-1)
        
#print(vertex_rename)
final_bet=defaultdict(list)
for i in range(0,be_tree):
    for j in range(0,len(bet[i])):
        if(visited_edge_list[bet[i][j]]==5+i):
            a1=bet[i][j][0]
            a2=bet[i][j][1]
            if(vertex_rename[i][a1]!=0):
                if(vertex_rename[i][a1]!=-1):
                    a1=vertex_rename[i][a1]
            if(vertex_rename[i][a2]!=0):
                if(vertex_rename[i][a2]!=-1):
                    a2=vertex_rename[i][a2]
            final_bet[i].append((a1,a2))
color_ind=-1
final_adj=defaultdict(list)
G2=nx.Graph()
for ind in range(0,be_tree):
    if(len(final_bet[ind])!=0):
        color_ind+=1
        print("Edges in Be Tree: ",ind)
        for col in range(0,len(final_bet[ind])):
            print(final_bet[ind][col])
            if(color_ind==0):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='r')
            elif(color_ind==1):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='b')
            elif(color_ind==2):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='g')
            elif(color_ind==3):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='k')
            elif(color_ind==4):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='w')
            elif(color_ind==5):
                G2.add_edge(final_bet[ind][col][0],final_bet[ind][col][1],color='m')
            final_degree[final_bet[ind][col][0]]+=1
            final_degree[final_bet[ind][col][1]]+=1
            final_adj[final_bet[ind][col][0]].append(final_bet[ind][col][1])
            final_adj[final_bet[ind][col][1]].append(final_bet[ind][col][0])
extra_label_nodes=[]
for i in range(1,curr_num):
    if(final_degree[i]==1 and i>nov):
        extra_labels+=1;
        extra_label_nodes.append(i)
edges=G2.edges()
colors=[G2[u][v]['color'] for u,v in edges]
nx.draw(G2,edges=edges,edge_color=colors,with_labels=True)
pos=nx.spring_layout(G2,k=0.15,iterations=20)           

print("No of betree: ", be_tree-1)
print("No of vertex split: ", vertex_split)
print("No of label swap: ", label_swap)
print("no.of edge-swap: ", edge_swap)
print("no of extra_labels: ", extra_labels)
#print("Final degree vertex which will be used for prufer sequence: ")
#for i in range(1,curr_num):
#    print("degree of node ",i," :",final_degree[i])
print("The nodes which should be stored for extra labels: ",extra_label_nodes)
curr_num-=1
print("To convert into prufer sequence we take node with id 1 as root node")
#to find parent of every node we have to run dfs once.
def final_dfs(src):
    global final_adj
    global final_visited
    global final_parent
    final_visited[src]=1;
    for i in range(len(final_adj[src])):
        if(final_visited[final_adj[src][i]]==-1):
            final_parent[final_adj[src][i]]=src
            final_dfs(final_adj[src][i])
final_dfs(1)
#for i in range(1,curr_num+1):
#    print("parent of node ",i," :",final_parent[i])
index=-1
for i in range(1,curr_num+1):
    if(final_degree[i]==1):
        index=i
        break;
prufer_sequence=[]
for i in range(0,curr_num-2):
    prufer_sequence.append(-1)
x=index
for j in range(0,curr_num-2):
    #print("x: ",x," index: ",index)
    if(final_parent[x]==-1):
        while(index<curr_num and final_degree[index+1]!=1):
            index+=1
        index+=1
        x=index 
    y=final_parent[x]
    prufer_sequence[j]=y
    final_degree[y]-=1
    if(y<index and final_degree[y]==1):
        x=y
    else:
        while(index<curr_num and final_degree[index+1]!=1):
            index+=1
        index+=1
        x=index
print("Prufer Sequence: ")
print(prufer_sequence)
    
        
            
        

    
            
                                    
                
                            
                        
            
                    
    
    